In [1]:
import aitk.robots
from aitk.utils import Grid
aitk.robots.__version__

'0.9.32'

This program allows the robot to wander around an empty world.
It uses a Grid imposed on the World to track the robot's location
over time. It plots a heat map of the locactions visted and computes 
the pecentage of Grid locations visited.

In [2]:
world = aitk.robots.World(width=200, height=200, scale=5.0)
robot = aitk.robots.Scribbler(x=100, y=100, a=87, max_trace_length=60)
robot.add_device(aitk.robots.RangeSensor(position=(6,-6),max=20,a=0,width=57.3,name="left-ir"))
robot.add_device(aitk.robots.RangeSensor(position=(6,6),max=20,a=0,width=57.3,name="right-ir"))
world.add_robot(robot)
world.update()
world.save()

Random seed set to: 7013779


In [3]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [4]:
from random import random

def wander(robot):
    left = robot[0].get_distance()
    right = robot[1].get_distance()
    x, y, direction = robot.get_pose()
    robot.state["grid"].update(x, y)
    robot.state["grid"].draw()
    # Save the robot's location only once per second
    if left == robot[0].get_max() and right == robot[1].get_max() and \
        robot.state["timer"] == 0:
        #front clear, move random direction
        direction = 1
        if random() < 0.5:
            direction = -1
        robot.move(0.5, direction*random())
        if robot.state["debug"]: robot.speak("F")
    elif robot.state["timer"] > 0 and robot.state["timer"] < 5:
        #timer triggered, continue current rotation
        robot.state["timer"] += 1
        if robot.state["debug"]: robot.speak("timer %d" % (robot.state["timer"]))
    elif left < robot[0].get_max():
        #obstacle on left, turn right, trigger timer
        robot.move(0, -0.4)
        robot.state["timer"] = 1
        if robot.state["debug"]: robot.speak("R")
    elif right < robot[1].get_max():
        #obstacle on right, turn left, trigger timer
        robot.move(0, 0.4)
        robot.state["timer"] = 1
        if robot.state["debug"]: robot.speak("L")
    else:
        #reset timer to zero
        robot.state["timer"] = 0
        if robot.state["debug"]: robot.speak("reset")    
    

In [5]:
### BUG HERE, re-running this cell should reset the Grid because
### we are re-creating the Grid from scratch, reusing the orignal
### world

from random import randrange

world.reset()
robot.state["timer"] = 0
robot.state["grid"] = Grid((10,10), world)
robot.state["debug"] = True

world.set_seed(randrange(1,100)) # allow randomness
world.seconds(60, [wander], real_time=False)

Using random seed: 7013779
Using random seed: 49


  0%|          | 0/600 [00:00<?, ?it/s]

Simulation stopped at: 00:01:00.0; speed 154.82 x real time


In [6]:
g = robot.state["grid"]
g.show()
g.analyze_visits()

  0   0   0   0   0   0   0   0   0   0 
  0   0   0   0  12  39   2  32  33   0 
  0   0   0   0  35  42  28  24  13   0 
  0   0   0   0  25  22  75  54   0   0 
  0   0   0   0  16  52  24  18  48   0 
  0   0   0   0   0   1   0   0   5   0 
  0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0 



0.21